In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
import atomap.api as am
import hyperspy.api as hs

In [2]:
#!pip install ipython --upgrade
#!pip install matplotlib --upgrade

In [2]:
s_whole = hs.load('whole half.hspy') 
s_1 = hs.load('first half.hspy') 
s_2 = hs.load('second half.hspy') 

In [3]:
s_whole.plot()

In [160]:
s_1.plot()
#s.axes_manager #pixel size = 0.02622468769550323nm

In [161]:
s_crop =s_1

In [162]:
s_crop.axes_manager
x_offset = s_crop.axes_manager[0].offset
y_offset = s_crop.axes_manager[1].offset
x_scale = s_crop.axes_manager[0].scale
y_scale = s_crop.axes_manager[1].scale

In [163]:
atom_lattice = am.load_atom_lattice_from_hdf5('atom_lattice_beta_whole2.hdf5')

In [165]:
sublattice_A = atom_lattice.sublattice_list[0]
sublattice_B = atom_lattice.sublattice_list[1]

In [485]:
atom_lattice.plot()

In [467]:
import numpy as np
blue_columns_x_ranges = [(3, 7), (38, 41), (27,30),(62,65)]
red_columns_x_ranges = [(8, 10), (43, 45),(22,25),(58,60)]
blue_ccolumns_x_ranges = [(14, 18), (49, 53)]
blue_coord = []
red_coord = []
blue_ccoord = []


# Function to slice the positions based on x range and calculate distances
def calculate_distances(sublattice_A, sublattice_B, blue_ranges, red_ranges,blue_ccolumns_x_ranges):
    distances_side = []
    distances_center = []  # Assuming you intend to use this for some calculations
    for blue_range, red_range in zip(blue_ranges, red_ranges):
        # Slice the x positions based on the given range
        blue_x = sublattice_B.x_position[(sublattice_B.x_position >= blue_range[0]) & (sublattice_B.x_position <= blue_range[1])]
        red_x = sublattice_A.x_position[(sublattice_A.x_position >= red_range[0]) & (sublattice_A.x_position <= red_range[1])]
        
        # Assuming that the y positions follow the same order as x positions
        blue_y = sublattice_B.y_position[(sublattice_B.x_position >= blue_range[0]) & (sublattice_B.x_position <= blue_range[1])]
        red_y = sublattice_A.y_position[(sublattice_A.x_position >= red_range[0]) & (sublattice_A.x_position <= red_range[1])]
        blue_coord.append((blue_x,blue_y))
        red_coord.append((red_x,red_y))
        # Calculate distances between all pairs of blue and red dots within the sliced range
        for bx, by in zip(blue_x, blue_y):
            for rx, ry in zip(red_x, red_y):
                distance = np.sqrt((bx - rx)**2 + (by - ry)**2)
                distances_side.append(distance)
        for center_range in blue_ccolumns_x_ranges:
            blue_center_x = sublattice_B.x_position[(sublattice_B.x_position >= center_range[0]) & (sublattice_B.x_position <= center_range[1])]
            blue_center_y = sublattice_B.y_position[(sublattice_B.x_position >= center_range[0]) & (sublattice_B.x_position <= center_range[1])]
            blue_ccoord.append((blue_center_x, blue_center_y))  # Correctly append tuples
            # Assuming we need to find the nearest red points for each blue center point
        for bx, by in zip(blue_center_x, blue_center_y):
            for r_x, r_y in zip(red_x, red_y):
                coord = find_nearest_two_points(bx, by, r_x, r_y)
                if coord is not None:
                    rx,ry = coord
                    distance = np.sqrt((bx - rx)**2 + (by - ry)**2)
                    distances_center.append(distance)

    return distances_side, distances_center

# Call the function and calculate distances
distances_side,distance_center = calculate_distances(sublattice_A, sublattice_B, blue_columns_x_ranges, red_columns_x_ranges,blue_ccolumns_x_ranges)
blue_x_1,blue_y_1 = blue_coord[0][0],blue_coord[0][1]
red_x_1,red_y_1  = red_coord[0][0],red_coord[0][1]
blue_x_2,blue_y_2 = blue_coord[1][0], blue_coord[1][1]
red_x_2,red_y_2  = red_coord[1][0],red_coord[1][1]
blue_x_3,blue_y_3 = blue_coord[2][0],blue_coord[2][1]
red_x_3,red_y_3  = red_coord[2][0],red_coord[2][1]
blue_x_4,blue_y_4 = blue_coord[3][0], blue_coord[3][1]
red_x_4,red_y_4  = red_coord[3][0],red_coord[3][1]
blue_cx_1,blue_cy_1 = blue_ccoord[0][0],blue_ccoord[0][1]
blue_cx_2,blue_cy_2 = blue_ccoord[1][0], blue_ccoord[1][1]

In [484]:
%matplotlib qt
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
image_height, image_width = s_crop.data.shape
scale_factor = 20 
fig_width = image_width / float(scale_factor)
fig_height = image_height / float(scale_factor)
plt.figure(figsize=(fig_width, fig_height))
def sort_dots(x, y):
    sorted_indices = np.argsort(y)
    sorted_x = x[sorted_indices]
    sorted_y = y[sorted_indices]
    return sorted_x, sorted_y

def distance(x1, x2, y1, y2):
    return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def plot_distances(blue_x, blue_y, red_x, red_y, color, x_scale):
    sorted_blue_x, sorted_blue_y = sort_dots(blue_x, blue_y)
    sorted_red_x, sorted_red_y = sort_dots(red_x, red_y)
    distance_list = []
    
    for bx, by, rx, ry in zip(sorted_blue_x, sorted_blue_y, sorted_red_x, sorted_red_y):
        plt.plot([bx, rx], [by, ry], color)
        distance_list.append(distance(bx, rx, by, ry) * x_scale)
    return distance_list


plt.figure(figsize=(fig_width, fig_height),dpi=300)
plt.imshow(s_crop.data[:115,:], cmap='gray')
plt.scatter(blue_x_1,blue_y_1,color='b',s=5)
plt.scatter(red_x_1,red_y_1,color='r',s=5)
plt.scatter(blue_x_2,blue_y_2,color='b',s=5)
plt.scatter(red_x_2,red_y_2,color='r',s=5)
plt.scatter(blue_x_3,blue_y_3,color='b',s=5)
plt.scatter(red_x_3,red_y_3,color='r',s=5)
plt.scatter(blue_x_4,blue_y_4,color='b',s=5)
plt.scatter(red_x_4,red_y_4,color='r',s=5)
plt.scatter(blue_cx_1, blue_cy_1, color='b',s=5)
plt.scatter(blue_cx_2, blue_cy_2, color='b',s=5)
distance_list_side = []
distance_list_side.extend(plot_distances(blue_x_1, blue_y_1, red_x_1, red_y_1, 'g-', x_scale))
distance_list_side.extend(plot_distances(blue_x_2, blue_y_2, red_x_2, red_y_2, 'g-', x_scale))
distance_list_side.extend(plot_distances(blue_x_3, blue_y_3, red_x_3, red_y_3, 'g-', x_scale))
distance_list_side.extend(plot_distances(blue_x_4, blue_y_4, red_x_4, red_y_4, 'g-', x_scale))
def plot_center_to_nearest_reds(blue_center_x, blue_center_y, red_x, red_y, color, x_scale):
    distance_list = []
    # Assuming red_x, red_y are the overall red dots, not sorted per segment as before
    #blue_center_x, blue_center_y = sort_dots(blue_center_x, blue_center_y)
    #red_x, red_y = sort_dots(red_x, red_y)
    for bcx, bcy in zip(blue_center_x, blue_center_y):
        for rx, ry in zip(red_x, red_y):
            d = distance(bcx, rx, bcy, ry)
            if bcx >rx and ry < bcy and d <16:
                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
                distance_list.append(d * x_scale)  # Scale and append the distance
                #distances.append(distance(bcx, rx, bcy, ry))
                #nearest_two_indices.append 
            elif bcx <rx and ry > bcy and d <16:

                plt.plot([bcx, rx], [bcy, ry], color)  # Plot line to each of the two nearest red dots
                distance_list.append(d* x_scale)  # Scale and append the distance
                #distances.append(distance(bcx, rx, bcy, ry))
        #nearest_two_indices = np.argsort(distances)[:2]  # Get indices of the nearest two red dots
        
        #for idx in nearest_two_indices:
       # plt.plot([bcx, red_x[idx]], [bcy, red_y[idx]], color)  # Plot line to each of the two nearest red dots
        #distance_list.append(distances[idx] * x_scale)  # Scale and append the distance

    return distance_list
red_x_1,red_y_1 =sort_dots(red_x_1,red_y_1 )
red_x_2,red_y_2 =sort_dots(red_x_2,red_y_2 )
red_x_3,red_y_3 =sort_dots(red_x_3,red_y_3 )
red_x_4,red_y_4 =sort_dots(red_x_4,red_y_4 )
all_red_x_1 = np.concatenate([red_x_1, red_x_3])
all_red_y_1 = np.concatenate([red_y_1, red_y_3])
all_red_x_2 = np.concatenate([red_x_2, red_x_4])
all_red_y_2 = np.concatenate([red_y_2, red_y_4])

distance_list_center = []
distance_list_center.extend(plot_center_to_nearest_reds(blue_cx_1, blue_cy_1, all_red_x_1, all_red_y_1, 'g-', x_scale))
distance_list_center.extend(plot_center_to_nearest_reds(blue_cx_2, blue_cy_2, all_red_x_2, all_red_y_2, 'g-', x_scale))
#plt.savefig('Figure1.tif')

In [479]:
i_points_In, i_record_In, p_record_In = sublattice_A.integrate_column_intensity(max_radius=3)
i_points_B, i_record_B, p_record_B = sublattice_B.integrate_column_intensity(max_radius=3)
fig, ax = plt.subplots()

i_record_In.plot(ax=ax,cmap='viridis',scalebar=False,alpha =0.50)
i_record_B.plot(ax=ax,cmap='viridis',scalebar=False,alpha =0.50)
s_crop.plot(ax=ax,cmap='gray',alpha =1)
plt.show()

Calculating Voronoi:   0%|          | 0/7910 [00:00<?, ?it/s]

Integrating:   0%|          | 0/34 [00:00<?, ?it/s]

Building intensity map:   0%|          | 0/7910 [00:00<?, ?it/s]

Calculating Voronoi:   0%|          | 0/7910 [00:00<?, ?it/s]

Integrating:   0%|          | 0/49 [00:00<?, ?it/s]

Building intensity map:   0%|          | 0/7910 [00:00<?, ?it/s]

In [478]:
masked_data_B = np.ma.masked_invalid(i_record_B.data)
masked_data_In = np.ma.masked_invalid(i_record_In.data)
cmap = plt.cm.viridis  
cmap.set_bad(color='none')
fig, ax = plt.subplots()
global_vmin = min(np.min(masked_data_B), np.min(masked_data_In))
global_vmax = max(np.max(masked_data_B), np.max(masked_data_In))

ax.imshow(s_crop.data, cmap='gray', alpha=1)
ax.imshow(masked_data_B, cmap=cmap,  vmin=global_vmin, vmax=global_vmax,alpha=0.6)  # fully opaque
ax.imshow(masked_data_In, cmap=cmap,vmin=global_vmin, vmax=global_vmax, alpha=0.6)
im = ax.imshow(masked_data_In, cmap=cmap, vmin=global_vmin, vmax=global_vmax, alpha=0.5)
cbar = fig.colorbar(im, ax=ax, orientation='vertical')
cbar.set_label('Intensity')

plt.show()

In [483]:
np.shape(s_crop.data)

(126, 70)

In [476]:
import statistics as st
def stat(distance):
    data = distance
    avg = sum(distance)/len(distance)
    std = st.stdev(data)
    count =len(distance)
    return {'data':data, 'average':avg,'stdev':std,'counts':count}
stats_result_center = stat(distance_list_center)
stats_str1 = "\n".join(f"{key}: {value}" for key, value in stats_result_center.items())
stats_result_side = stat(distance_list_side)
stats_str2 = "\n".join(f"{key}: {value}" for key, value in stats_result_side.items())
# Save the string to a file
file_path = 'center_distance_statistics.txt'  # Specify your desired file path
with open(file_path, 'w') as file:
    file.write(stats_str1)
# Save the string to a file
file_path2 = 'side_distance_statistics.txt'  # Specify your desired file path
with open(file_path2, 'w') as file:
    file.write(stats_str2)

In [477]:
In_Se_ratio = np.average(i_points_In) / np.average(i_points_B)
std_normalized_In = np.std(i_points_In) / np.average(i_points_In)
std_normalized_B = np.std(i_points_B) / np.average(i_points_B)
data_to_save = np.array([In_Se_ratio, std_normalized_In, std_normalized_B])
np.savetxt('Beta In_Se_ratio, StdDev_In_to_Avg_In, StdDev_B_to_Avg_B.txt',data_to_save, header='In_Se_ratio, StdDev_In/Avg_In, StdDev_B/Avg_B')
np.savetxt('Beta In_intensity.txt',i_points_In)
np.savetxt('Beta B_intensity.txt',i_points_B)

In [382]:
s_crop2 = s_2

In [395]:
atom_lattice2 = am.load_atom_lattice_from_hdf5('atom_lattice-2.hdf5')

In [398]:
sublattice_In2 = atom_lattice2.sublattice_list[0]
sublattice_B2 = atom_lattice2.sublattice_list[1]

In [399]:
blue_columns_x_ranges2 = [(4, 6)]
red_columns_x_ranges2 = [(7, 11)]
blue_coord2 = []
red_coord2 = []
# Function to slice the positions based on x range and calculate distances
def calculate_distances(sublattice_A, sublattice_B, blue_ranges, red_ranges):
    distances = []
    for blue_range, red_range in zip(blue_ranges, red_ranges):
        # Slice the x positions based on the given range
        blue_x = sublattice_B.x_position[(sublattice_B.x_position >= blue_range[0]) & (sublattice_B.x_position <= blue_range[1])]
        red_x = sublattice_A.x_position[(sublattice_A.x_position >= red_range[0]) & (sublattice_A.x_position <= red_range[1]) 
                & (sublattice_A.y_position > 5)]
        
        # Assuming that the y positions follow the same order as x positions
        blue_y = sublattice_B.y_position[(sublattice_B.x_position >= blue_range[0]) & (sublattice_B.x_position <= blue_range[1])]
        red_y = sublattice_A.y_position[(sublattice_A.x_position >= red_range[0]) & (sublattice_A.x_position <= red_range[1])
                                       & (sublattice_A.y_position > 5)]
        blue_coord2.append((blue_x,blue_y))
        red_coord2.append((red_x,red_y))
        # Calculate distances between all pairs of blue and red dots within the sliced range
        for bx, by in zip(blue_x, blue_y):
            for rx, ry in zip(red_x, red_y):
                distance = np.sqrt((bx - rx)**2 + (by - ry)**2)
                distances.append(distance)
    return distances


# Call the function and calculate distances
distances2 = calculate_distances(sublattice_In2, sublattice_B2, blue_columns_x_ranges2, red_columns_x_ranges2)
blue2_x_1,blue2_y_1 = blue_coord2[0][0],blue_coord2[0][1]
red2_x_1,red2_y_1  = red_coord2[0][0],red_coord2[0][1]


plt.imshow(s_crop2.data, cmap='gray')
plt.scatter(blue2_x_1,blue2_y_1,color='b' )
plt.scatter(red2_x_1,red2_y_1,color='r' )
distance_list2 = []
distance_list2.extend(plot_distances(blue2_x_1, blue2_y_1, red2_x_1, red2_y_1, 'g-', x_scale))
#plt.savefig('Figure2.tif')

In [419]:
print('data = ',distance_list2,'count =',len(distance_list2),f'average = {round(statistics.mean(distance_list2),3)}nm',f'std = {round(statistics.stdev(distance_list2),3)}nm')
info2 = f'data = {distance_list2}  count = 8 average = 0.123nm std = 0.009nm'
np.savetxt('2nd distance',distance_list2)
file_path2 = '2rd distance_statistics.txt'  # Specify your desired file path
with open(file_path2, 'w') as file:
    file.write(info2)

data =  [0.1499611379627792, 0.1604760542771656, 0.1629218524245166, 0.14182839644861409, 0.15163586808426405, 0.14358744271105697, 0.15055280808417085, 0.14470868569934056] count = 8 average = 0.151nm std = 0.008nm


In [422]:
blue_columns_x_ranges3 = (16,20),(29,32)
red_columns_x_ranges3 = (20,23),(33,36)
blue_coord3 = []
red_coord3 = []
# Function to slice the positions based on x range and calculate distances
def calculate_distances(sublattice_A, sublattice_B, blue_ranges, red_ranges):
    distances = []
    for blue_range, red_range in zip(blue_ranges, red_ranges):
        # Slice the x positions based on the given range
        blue_x = sublattice_B.x_position[(sublattice_B.x_position >= blue_range[0]) & (sublattice_B.x_position <= blue_range[1])]
        red_x = sublattice_A.x_position[(sublattice_A.x_position >= red_range[0]) & (sublattice_A.x_position <= red_range[1]) 
                & (sublattice_A.y_position > 5)]
        
        # Assuming that the y positions follow the same order as x positions
        blue_y = sublattice_B.y_position[(sublattice_B.x_position >= blue_range[0]) & (sublattice_B.x_position <= blue_range[1])]
        red_y = sublattice_A.y_position[(sublattice_A.x_position >= red_range[0]) & (sublattice_A.x_position <= red_range[1])
                                       & (sublattice_A.y_position > 5)]
        blue_coord3.append((blue_x,blue_y))
        red_coord3.append((red_x,red_y))
        # Calculate distances between all pairs of blue and red dots within the sliced range
        for bx, by in zip(blue_x, blue_y):
            for rx, ry in zip(red_x, red_y):
                distance = np.sqrt((bx - rx)**2 + (by - ry)**2)
                distances.append(distance)
    return distances


# Call the function and calculate distances
distances3 = calculate_distances(sublattice_In2, sublattice_B2, blue_columns_x_ranges3, red_columns_x_ranges3)
blue3_x_1,blue3_y_1 = blue_coord3[0][0],blue_coord3[0][1]
red3_x_1,red3_y_1  = red_coord3[0][0],red_coord3[0][1]
blue3_x_2,blue3_y_2 = blue_coord3[1][0],blue_coord3[1][1]
red3_x_2,red3_y_2  = red_coord3[1][0],red_coord3[1][1]

In [434]:
plt.imshow(s_crop2.data, cmap='gray')
plt.scatter(blue3_x_1,blue3_y_1,color='y' )
plt.scatter(red3_x_1,red3_y_1,color='y' )
plt.scatter(blue3_x_2,blue3_y_2,color='y' )
plt.scatter(red3_x_2,red3_y_2,color='y' )
distance_list3 = []
distance_list3.extend(plot_distances(blue3_x_1, blue3_y_1, red3_x_1, red3_y_1, 'g-', x_scale))
distance_list3.extend(plot_distances(blue3_x_2, blue3_y_2, red3_x_2, red3_y_2, 'g-', x_scale))
#plt.savefig('Figure3.tif')

In [427]:
print('data = ',distance_list3[:8],'count =',len(distance_list3[:8]),f'average = {round(statistics.mean(distance_list3[:8]),3)}nm',
      f'std = {round(statistics.stdev(distance_list3[:8]),3)}nm')
print('data = ',distance_list3[8:],'count =',len(distance_list3[8:]),f'average = {round(statistics.mean(distance_list3[8:]),3)}nm',
      f'std = {round(statistics.stdev(distance_list3[8:]),3)}nm')
info3 = f'data = {distance_list3[:8]}  count = 8 average = 0.123nm std = 0.009nm \ndata = {distance_list3[8:]} count = 9 average = 0.142nm std = 0.008nm'
np.savetxt('3rd distance',distance_list3)
file_path3 = '3rd distance_statistics.txt'  # Specify your desired file path
with open(file_path3, 'w') as file:
    file.write(info3)

data =  [0.14017968046018148, 0.12255381556283934, 0.11449556107474113, 0.12387059606811107, 0.11079780648367703, 0.12285144984511613, 0.12620620306276917, 0.12183309550638209] count = 8 average = 0.123nm std = 0.009nm
data =  [0.13108966867924476, 0.14333513356535243, 0.1485622057127049, 0.1428913593455131, 0.13899486654066123, 0.1536490849187982, 0.14439615429012542, 0.12769656294096027, 0.14613513390999236] count = 9 average = 0.142nm std = 0.008nm


In [435]:
i_points_In2, i_record_In2, p_record_In2 = sublattice_In2.integrate_column_intensity(max_radius=3)
i_points_B2, i_record_B2, p_record_B2 = sublattice_B2.integrate_column_intensity(max_radius=3)
fig, ax = plt.subplots()
i_record_In2.plot(ax=ax,cmap='viridis',scalebar=False)
i_record_B2.plot(ax=ax,cmap='viridis',scalebar=False)
s_crop2.plot(ax=ax,cmap='gray',alpha =1)
plt.show()

Calculating Voronoi:   0%|          | 0/5040 [00:00<?, ?it/s]

Integrating:   0%|          | 0/26 [00:00<?, ?it/s]

Building intensity map:   0%|          | 0/5040 [00:00<?, ?it/s]

Calculating Voronoi:   0%|          | 0/5040 [00:00<?, ?it/s]

Integrating:   0%|          | 0/25 [00:00<?, ?it/s]

Building intensity map:   0%|          | 0/5040 [00:00<?, ?it/s]

In [443]:
masked_data_B2 = np.ma.masked_invalid(i_record_B2.data)
masked_data_In2 = np.ma.masked_invalid(i_record_In2.data)
cmap2 = plt.cm.viridis  
cmap2.set_bad(color='none')
fig2, ax2 = plt.subplots()
global_vmin2 = min(np.min(masked_data_B2), np.min(masked_data_In2))
global_vmax2 = max(np.max(masked_data_B2), np.max(masked_data_In2))

ax2.imshow(s_crop2.data, cmap='gray', alpha=1)
ax2.imshow(masked_data_B2, cmap=cmap2,  vmin=global_vmin2, vmax=global_vmax2,alpha=0.6)  # fully opaque
ax2.imshow(masked_data_In2, cmap=cmap2,vmin=global_vmin2, vmax=global_vmax2, alpha=0.6)
im2 = ax.imshow(masked_data_In2, cmap=cmap2, vmin=global_vmin2, vmax=global_vmax2, alpha=0.5)
cbar2 = fig.colorbar(im2, ax=ax2, orientation='vertical')
cbar2.set_label('Intensity')

plt.show()

In [437]:
In_Se_ratio2 = np.average(i_points_In2) / np.average(i_points_B2)
std_normalized_In2 = np.std(i_points_In2) / np.average(i_points_In2)
std_normalized_B2 = np.std(i_points_B2) / np.average(i_points_B2)
data_to_save2 = np.array([In_Se_ratio2, std_normalized_In2, std_normalized_B2])
np.savetxt('Zigzag_In_Se_ratio, StdDev_In_to_Avg_In, StdDev_B_to_Avg_B.txt',data_to_save2, header='In_Se_ratio, StdDev_In/Avg_In, StdDev_B/Avg_B')
np.savetxt('Zigzag_In_intensity.txt',i_points_In2)
np.savetxt('Zigzag_intensity.txt',i_points_B2)

In [438]:
s_whole.plot()

In [442]:
plt.imshow(s_whole.data, cmap='gray')
plt.scatter(blue_x_1,blue_y_1,color='b' )
plt.scatter(red_x_1,red_y_1,color='r' )
plt.scatter(blue_x_2,blue_y_2,color='b' )
plt.scatter(red_x_2,red_y_2,color='r' )
plt.scatter(blue_x_3,blue_y_3,color='b' )
plt.scatter(red_x_3,red_y_3,color='r' )
plt.scatter(blue_x_4,blue_y_4,color='b' )
plt.scatter(red_x_4,red_y_4,color='r' )
plt.scatter(blue_cx_1, blue_cy_1, color='b')
plt.scatter(blue_cx_2, blue_cy_2, color='b')
distance_list = []
plot_distances(blue_x_1, blue_y_1, red_x_1, red_y_1, 'g-', x_scale)
plot_distances(blue_x_2, blue_y_2, red_x_2, red_y_2, 'g-', x_scale)
plot_distances(blue_x_3, blue_y_3, red_x_3, red_y_3, 'g-', x_scale)
plot_distances(blue_x_4, blue_y_4, red_x_4, red_y_4, 'g-', x_scale)

shifted =155-85

plt.scatter(blue2_x_1+shifted,blue2_y_1,color='b' )
plt.scatter(red2_x_1+shifted,red2_y_1,color='r' )
plot_distances(blue2_x_1+shifted, blue2_y_1, red2_x_1+shifted, red2_y_1, 'g-', x_scale)

plt.scatter(blue3_x_1+shifted,blue3_y_1,color='b' )
plt.scatter(red3_x_1+shifted,red3_y_1,color='r' )
plt.scatter(blue3_x_2+shifted,blue3_y_2,color='b' )
plt.scatter(red3_x_2+shifted,red3_y_2,color='r' )
plot_distances(blue3_x_1+shifted, blue3_y_1, red3_x_1+shifted, red3_y_1, 'g-', x_scale)
plot_distances(blue3_x_2+shifted, blue3_y_2, red3_x_2+shifted, red3_y_2, 'g-', x_scale)
plot_center_to_nearest_reds(blue_cx_1, blue_cy_1, all_red_x_1, all_red_y_1, 'g-', x_scale)
plot_center_to_nearest_reds(blue_cx_2, blue_cy_2, all_red_x_2, all_red_y_2, 'g-', x_scale)
#plt.savefig('Figure_whole.tif')

[0.22574782864213458,
 0.21335813389981417,
 0.23017762932065342,
 0.22023172065084456,
 0.2327115764373473,
 0.231004215003355,
 0.23553672562186628,
 0.222460961231442,
 0.2242009740463621,
 0.2285543921868832,
 0.24225137219144158,
 0.22135634009477498,
 0.21605138901665982,
 0.23611218598335265,
 0.22069814831819445,
 0.22697292546758902]

In [496]:
masked_data_B = np.ma.masked_invalid(i_record_B.data)
masked_data_In = np.ma.masked_invalid(i_record_In.data)
cmap = plt.cm.viridis  
cmap.set_bad(color='none')
fig, ax = plt.subplots()
global_vmin = min(np.min(masked_data_B), np.min(masked_data_In),np.min(masked_data_B2), np.min(masked_data_In2))
global_vmax = max(np.max(masked_data_B), np.max(masked_data_In),np.max(masked_data_B2), np.max(masked_data_In2))

ax.imshow(s_whole.data, cmap='gray', alpha=1)
masked_data_B_whole = np.concatenate((masked_data_B,masked_data_B2[:113,:]), axis = 1)
masked_data_In_whole = np.concatenate((masked_data_In,masked_data_In2[:113,:]), axis = 1)
ax.imshow(masked_data_B_whole, cmap=cmap,  vmin=global_vmin, vmax=global_vmax,alpha=0.6)  # fully opaque
ax.imshow(masked_data_In_whole, cmap=cmap,vmin=global_vmin, vmax=global_vmax, alpha=0.6)
#im = ax.imshow(masked_data_In, cmap=cmap, vmin=global_vmin, vmax=global_vmax, alpha=0.5)


#masked_data_B2 = np.ma.masked_invalid(i_record_B2.data)
#masked_data_In2 = np.ma.masked_invalid(i_record_In2.data)

#ax.imshow(masked_data_B2, cmap=cmap2,  vmin=global_vmin2, vmax=global_vmax2,alpha=0.6)  # fully opaque
#ax.imshow(masked_data_In2, cmap=cmap2,vmin=global_vmin2, vmax=global_vmax2, alpha=0.6)
im = ax.imshow(masked_data_In_whole, cmap=cmap, vmin=global_vmin, vmax=global_vmax, alpha=0.5)
cbar = fig.colorbar(im, ax=ax, orientation='vertical')
cbar.set_label('Intensity')

plt.show()

In [493]:
print (np.shape(s_whole.data[:113,:]), np.shape(masked_data_In))

(113, 110) (113, 70)
